In [1]:
import cv2
import numpy as np
import dlib
from scipy.spatial import distance 
import math
from imutils import face_utils
import pandas as pd
import joblib
import winsound
from datetime import datetime
import time

In [2]:
# Load pre-trained model
model = joblib.load('knn_model.pkl')
frequency=1500
duration=1500
# Define labels
labels = {0: 'Closed', 1: 'Open'}

In [3]:
# Using a pre-trained model to extract face landmarks
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [4]:
# Defining function to get eye_aspect ratio: ear
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [5]:
#Defining function to get circularity of the eye
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [6]:
def model(landmarks):

    features = pd.DataFrame(columns=["EAR","Circularity"])
    eye = landmarks[36:68]
    ear = eye_aspect_ratio(eye)
    cir = circularity(eye)

    new_row = ({"EAR":ear,"Circularity": cir})
    features.loc[len(features)] = new_row
    df=np.asarray(features)
    model = joblib.load('knn_model.pkl')
    Result = model.predict(df)
    label_index=np.argmax(Result)
    label=labels[label_index]
    Result_String = label
    

    return Result_String, df

In [7]:
# Open video capture
cap = cv2.VideoCapture(0)  
# Define the number of seconds until Sleep Alert and sound is used
seconds=10
counter=0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects=detector(frame, 0)
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        Result_String, features = model(shape)
        if Result_String == 'Open':
            cv2.putText(frame, "Alert", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            counter=0
        
        if Result_String == 'Closed':
            counter=counter+1
            
            if counter > seconds:
                cv2.putText(frame, "Sleeping!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                winsound.Beep(frequency, duration)
            else:
                cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        # Draw on our image, all the finded cordinate points (x,y) 
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        # Show the image
    cv2.imshow("Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()